<a href="https://colab.research.google.com/github/duytrung/QuanLyTuyenSinh/blob/main/T%C3%A0i_li%E1%BB%87u_bu%E1%BB%95i_9(1)_Python_with_Google_Sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📝 Google Sheets Editor with Streamlit - Giải Thích Chi Tiết**

**🖥️ Source code:** https://github.com/riodev1310/python_automation_scripts/blob/main/google_sheet.py

## **🚀 Giới Thiệu**

Chương trình này sử dụng `Streamlit` để tạo giao diện chỉnh sửa Google Sheets. Nó kết nối với Google Sheets API thông qua `gspread` và xác thực bằng `google.oauth2.service_account.Credentials`. Chương trình cung cấp các chức năng như tải dữ liệu từ Google Sheets, tìm kiếm, cập nhật, xóa và thêm dữ liệu mới.

## **1. 🔧 Cấu hình Google Sheets API**

In [ ]:
import streamlit as st
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

📌 Import thư viện cần thiết:

+ `streamlit`: Tạo giao diện người dùng.

+ `pandas`: Xử lý dữ liệu dạng bảng.

+ `gspread`: Làm việc với Google Sheets API.

+ `Credentials từ google.oauth2.service_account`: Xác thực với Google Sheets.

In [ ]:
creds = Credentials.from_service_account_file("google_sheet_key.json", scopes=["https://www.googleapis.com/auth/spreadsheets"])

🔑 Đọc thông tin xác thực từ file JSON để kết nối với Google Sheets API.

🔍 scopes: Quyền truy cập vào Google Sheets.

In [ ]:
client = gspread.authorize(creds)

🔗 Sử dụng gspread để tạo kết nối với Google Sheets.

## **2. 📊 Hàm Tương Tác với Google Sheets**

### **2.1. 📥 Hàm Lấy Dữ Liệu**

In [ ]:
def get_sheet(SHEET_ID, SHEET_NAME):
    sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
    data = sheet.get_all_records()
    return pd.DataFrame(data) if data else pd.DataFrame()

📄 Mở Google Sheet dựa trên SHEET_ID và lấy worksheet theo SHEET_NAME.

🔄 Lấy toàn bộ dữ liệu dưới dạng danh sách từ Google Sheets.

📑 Chuyển dữ liệu thành DataFrame của pandas để dễ xử lý.

### **2.2. 💾 Hàm Lưu Dữ Liệu**

In [ ]:
def save_to_sheet(df, SHEET_ID, SHEET_NAME):
    sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
    sheet.clear()
    sheet.update([df.columns.values.tolist()] + df.values.tolist())

❌ Xóa toàn bộ dữ liệu cũ (sheet.clear()).

✅ Ghi dữ liệu mới từ DataFrame vào Google Sheets.

### **2.3. 🔎 Hàm Tìm Kiếm**

In [ ]:
def search_data(df, column, query):
    return df[df[column].astype(str).str.contains(str(query), case=False, na=False)]

🔍 Tìm kiếm giá trị query trong column.

🔄 Chuyển tất cả dữ liệu về kiểu chuỗi (astype(str)) để tránh lỗi.

🔠 case=False: Không phân biệt chữ hoa/thường.

🚫 na=False: Xử lý giá trị thiếu (NaN).

### **2.4. 🗑️ Hàm Xóa Dữ Liệu**

In [ ]:
def delete_searched_data(df, column, query, SHEET_ID, SHEET_NAME):
    df = df[~df[column].astype(str).str.contains(str(query), case=False, na=False)]
    save_to_sheet(df, SHEET_ID, SHEET_NAME)
    return df

❌ Lọc ra những hàng không khớp với giá trị tìm kiếm (~ là phủ định).

📂 Lưu lại dữ liệu đã cập nhật vào Google Sheets.

### **2.5. ✏️ Hàm Cập Nhật Dữ Liệu**

In [ ]:
def update_data_by_column(df, search_column, search_value, update_column, new_value, SHEET_ID, SHEET_NAME):
    mask = df[search_column].astype(str).str.contains(str(search_value), na=False)
    if not mask.any():
        st.warning("No matching records found to update.")
        return False, df
    df.loc[mask, update_column] = convert_value(new_value, df[update_column].dtype)
    save_to_sheet(df, SHEET_ID, SHEET_NAME)
    return True, df

🔍 Tìm các hàng chứa search_value.

⚠️ Nếu không có kết quả, hiển thị cảnh báo.

✏️ Cập nhật giá trị của update_column.

📂 Lưu dữ liệu cập nhật vào Google Sheets.

## **3. 🖥️ Xây Dựng Giao Diện Streamlit**

In [ ]:
st.title("📊 Google Sheets Editor with Streamlit")

🎯 Hiển thị tiêu đề ứng dụng.

### **3.1. 📝 Nhập Thông Tin Google Sheets**

In [ ]:
SHEET_ID = st.text_input("Enter sheet's ID: ")
SHEET_NAME = st.text_input("Enter sheet's name: ")

⌨️ Nhập SHEET_ID và SHEET_NAME từ người dùng.

### **3.2. 📃 Hiển Thị Dữ Liệu**

In [ ]:
if SHEET_ID and SHEET_NAME:
    df = get_sheet(SHEET_ID, SHEET_NAME)
    if df.empty:
        st.warning("No data found in Google Sheet.")
    else:
        st.dataframe(df)

⌨️ Nhập SHEET_ID và SHEET_NAME từ người dùng.

### **3.2. 📃 Hiển Thị Dữ Liệu**

In [ ]:
if SHEET_ID and SHEET_NAME:
    df = get_sheet(SHEET_ID, SHEET_NAME)
    if df.empty:
        st.warning("No data found in Google Sheet.")
    else:
        st.dataframe(df)

📊 Nếu có ID và tên sheet, tải dữ liệu lên.

⚠️ Hiển thị cảnh báo nếu không có dữ liệu.

## **📌 Tổng Kết**

📌 Ứng dụng này giúp thao tác với Google Sheets một cách dễ dàng thông qua Streamlit, hỗ trợ tìm kiếm, chỉnh sửa, xóa và cập nhật dữ liệu. Nó có thể mở rộng để xử lý nhiều tập dữ liệu phức tạp hơn.